In [1]:
import pandas as pd
import numpy as np

import yfinance as yf
import plotly.graph_objects as go

In [2]:
## Fetch Data

ticker_data = yf.download("EURNZD=X", period="3y", interval="1d")

ticker_data = ticker_data[["Open", "Close", "Low", "High"]]

[*********************100%***********************]  1 of 1 completed


In [5]:
class Backtest:
    balance = 0

    stoploss = 0
    target = 0
    
    position = {
    "entry_date": None,
    "entry_price": None,
    "exit_price": None,
    "exit_date": None,
    "target": None,
    "stoploss": None,
    "PnL": None,
    "crossover_type": None,
    "active": False,
    "size": None,
    "cost": None,
    "balance": None,
}

    trade_book = pd.DataFrame(
        columns=[
            "entry_date",
            "entry_price",
            "exit_price",
            "exit_date",
            "target",
            "stoploss",
            "PnL",
            "crossover_type",
            "active",
            "size",
            "balance",
        ]
    )

    def __init__(self, balance, stoploss, reward_ratio):
        self.balance = balance
        self.stoploss = stoploss
        self.target = stoploss * reward_ratio
        
        self.clear_position()

    def clear_position(self):
        self.position["entry_date"] = None
        self.position["entry_price"] = None
        self.position["exit_price"] = None
        self.position["exit_date"] = None

        self.position["target"] = None
        self.position["stoploss"] = None

        self.position["PnL"] = None
        self.position["crossover_type"] = None

        self.position["active"] = False
        self.position["size"] = 0
        self.position["balance"] = None

        self.position["status"] = None

    def open(self, entry_date, entry_price, stoploss, target, size, crossover_type):
        self.position["active"] = True

        self.position["entry_date"] = entry_date
        self.position["entry_price"] = entry_price

        self.position["stoploss"] = stoploss
        self.position["target"] = target

        self.position["size"] = size
        self.position["crossover_type"] = crossover_type
        
    def close(self, candle, exit_price, exit_date):
        # Closes the trade / position
        self.position["active"] = False
        self.position["exit_date"] = candle.name
        self.position["exit_price"] = exit_price

        row = pd.Series(self.position)
        self.trade_book.loc[len(self.trade_book)] = row

        self.clear_position()
        
    def strategy ():
        ## to be implemented in subclass
        pass
    
    def print(self):
        print(self.balance, self.stoploss, self.target)
    

In [6]:
class Tester(Backtest):
    ticker_data = None

    def __init__(self, balance, stoploss, reward_ratio):
        super().__init__(balance, stoploss, reward_ratio)

    def init(self, ticker_data, small_ema, large_ema):
        ticker_data["small_ema"] = (
            ticker_data["Close"].ewm(span=small_ema, adjust=False).mean()
        )
        ticker_data["large_ema"] = (
            ticker_data["Close"].ewm(span=large_ema, adjust=False).mean()
        )

        ticker_data["crossover_pos"] = np.where(
            ticker_data["small_ema"] > ticker_data["large_ema"], 1, 0
        )
        ticker_data["crosspoint"] = np.where(
            ticker_data["crossover_pos"] != ticker_data["crossover_pos"].shift(1),
            True,
            False,
        )
        
        self.ticker_data = None
        self.ticker_data = ticker_data
        
    def chart(self, small_ema, large_ema):
        fig = go.Figure(
            data=[
                go.Candlestick(
                    x=ticker_data.index,
                    open=ticker_data["Open"],
                    high=ticker_data["High"],
                    low=ticker_data["Low"],
                    close=ticker_data["Close"],
                ),
                go.Scatter(
                    x=ticker_data.index,
                    y=ticker_data["small_ema"],
                    name=f"small_ema_{small_ema}",
                ),
                go.Scatter(
                    x=ticker_data.index,
                    y=ticker_data["large_ema"],
                    name=f"large_ema_{large_ema}",
                ),
            ]
        )


        fig.update_layout(xaxis_rangeslider_visible=False)
        fig.show()

    def print(self):
        print(self.position)

    def run(self):
        if self.ticker_data is None:
            raise Exception("No ticker data to run backtest on")
        
        start_index = 0
        
        
        while start_index < len(self.ticker_data):
            self.strategy(self.ticker_data.iloc[start_index])
            start_index += 1
            
    def stats(self):
        pass

    def strategy(self, candle):
        
        print("Candle \n\n", candle.name)
        
        if self.position["active"] == True:
            # check if target met or stoploss hit
            # calculate trade results, PnL
            # close position
            
            print(self.position)
            
            if self.position['crossover_type'] == "long":
                
                if candle['Close'] < self.position['stoploss']:
                    self.close(candle, self.position['stoploss'], candle.name)
                    return
                
                if candle['Close'] > self.position['target']:
                    self.close(candle, self.position['target'], candle.name)
                    return
                
                
            if self.position['crossover_type'] == "short":
                
                if candle['Close'] > self.position['stoploss']:
                    self.close(candle, self.position['stoploss'], candle.name)
                    return
                
                if candle['Close'] < self.position['target']:
                    self.close(candle, self.position['target'], candle.name)
                    return
            
            return
        
        print(self.trade_book)

        if candle["crosspoint"] == True:
            entry_price = candle["Open"]
            stoploss = entry_price - self.stoploss
            target = entry_price + self.target
            
            crossover_type = "long" if candle["crossover_pos"] == 1 else "short";

            self.open(
                entry_date=candle.name,
                entry_price=entry_price,
                stoploss=stoploss,
                target=target,
                size=10,
                crossover_type = crossover_type,
            )

ttr = Tester(100, 0.01, 3)

ttr.init(ticker_data, 50, 200)
ttr.chart(55, 200)
ttr.run()

t_book2= ttr.trade_book

Candle 

 2020-04-07 00:00:00
Empty DataFrame
Columns: [entry_date, entry_price, exit_price, exit_date, target, stoploss, PnL, crossover_type, active, size, balance]
Index: []
Candle 

 2020-04-08 00:00:00
{'entry_date': Timestamp('2020-04-07 00:00:00'), 'entry_price': 1.8157199621200562, 'exit_price': None, 'exit_date': None, 'target': 1.8457199621200562, 'stoploss': 1.8057199621200561, 'PnL': None, 'crossover_type': 'short', 'active': True, 'size': 10, 'cost': None, 'balance': None, 'status': None}
Candle 

 2020-04-09 00:00:00
  entry_date entry_price exit_price  exit_date   target stoploss   PnL   
0 2020-04-07     1.81572    1.80572 2020-04-08  1.84572  1.80572  None  \

  crossover_type active size balance  
0          short  False   10    None  
Candle 

 2020-04-10 00:00:00
  entry_date entry_price exit_price  exit_date   target stoploss   PnL   
0 2020-04-07     1.81572    1.80572 2020-04-08  1.84572  1.80572  None  \

  crossover_type active size balance  
0          short  F